# Final Project - Part 2
 - Mohnish Tarak Bhagat (mbhagat3)
 - Ruthvik Bonthapally (rbonth2)
 
 ## Table of Content
* [Importing and Synthesizing Dataset](#first-bullet)
* [Interactive Dashboard](#second-bullet)
* [Explaination and Further Scope](#third-bullet)

## Importing and Synthesizing Dataset <a class="anchor" id="first-bullet"></a>

In [1]:
%matplotlib inline
import cartopy
import pandas as pd
import matplotlib.pyplot as plt
import geopandas 
import ipyleaflet
import numpy as np
import bqplot
import ipywidgets
import datetime

In [2]:
def get_ids_and_names(states_map):
    ids = []
    state_names = []
    state_data_vec = states_map.map_data['objects']['subunits']['geometries']
    for i in range(len(state_data_vec)):
        if state_data_vec[i]['properties'] is not None:
            state_names.append(state_data_vec[i]['properties']['name'])
            ids.append(state_data_vec[i]['id'])
    return np.array(ids), np.array(state_names)


In [3]:
abbrev_to_us_state = {
 'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming',
 'DC': 'District of Columbia',
 'AS': 'American Samoa',
 'GU': 'Guam',
 'MP': 'Northern Mariana Islands',
 'PR': 'Puerto Rico',
 'UM': 'United States Minor Outlying Islands',
 'VI': 'U.S. Virgin Islands'}

In [4]:
data = pd.read_csv("loans_full_schema.csv")

In [5]:
data["months_since_last_delinq"] = data["months_since_last_delinq"].fillna(0)

In [6]:
data['state'] = data['state'].replace(list(abbrev_to_us_state.keys()),list(abbrev_to_us_state.values()))

In [7]:
data.head()

,emp_title,emp_length,state,homeownership,annual_income,verified_income,debt_to_income,annual_income_joint,verification_income_joint,debt_to_income_joint,...,sub_grade,issue_month,loan_status,initial_listing_status,disbursement_method,balance,paid_total,paid_principal,paid_interest,paid_late_fees
0,global config engineer,3.0,New Jersey,MORTGAGE,90000.0,Verified,18.01,NaN,NaN,NaN,...,C3,Mar-2018,Current,whole,Cash,27015.86,1999.33,984.14,1015.19,0.0
1,warehouse office clerk,10.0,Hawaii,RENT,40000.0,Not Verified,5.04,NaN,NaN,NaN,...,C1,Feb-2018,Current,whole,Cash,4651.37,499.12,348.63,150.49,0.0
2,assembly,3.0,Wisconsin,RENT,40000.0,Source Verified,21.15,NaN,NaN,NaN,...,D1,Feb-2018,Current,fractional,Cash,1824.63,281.80,175.37,106.43,0.0
3,customer service,1.0,Pennsylvania,RENT,30000.0,Not Verified,10.16,NaN,NaN,NaN,...,A3,Jan-2018,Current,whole,Cash,18853.26,3312.89,2746.74,566.15,0.0
4,security supervisor,10.0,California,RENT,35000.0,Verified,57.96,57000.0,Verified,37.66,...,C3,Mar-2018,Current,whole,Cash,21430.15,2324.65,1569.85,754.80,0.0


In [8]:
table = pd.pivot_table(data, values='loan_amount', index=['state'], columns=['loan_purpose'], aggfunc=np.sum)
table.reset_index(inplace=True)
table.head()

loan_purpose,state,car,credit_card,debt_consolidation,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation
0,Alabama,NaN,347100.0,1275125.0,157725.0,11700.0,10300.0,17600.0,10000.0,107000.0,NaN,10000.0,NaN
1,Alaska,18000.0,160750.0,350300.0,40000.0,NaN,NaN,33000.0,NaN,30800.0,NaN,NaN,NaN
2,Arizona,12100.0,857750.0,2132700.0,486325.0,NaN,210000.0,25300.0,1000.0,255900.0,NaN,132400.0,NaN
3,Arkansas,6600.0,181850.0,630225.0,60400.0,NaN,16500.0,23200.0,NaN,43500.0,NaN,15000.0,2500.0
4,California,212750.0,4772700.0,11642275.0,1202000.0,310600.0,670150.0,360600.0,67200.0,2295725.0,5000.0,415200.0,119500.0


In [9]:
purpose = list(table.columns.values)
purpose = np.array(purpose[1:])
purpose1 = purpose.astype('str')
purpose = [i for i in range(len(purpose))]
purpose1

array(['car', 'credit_card', 'debt_consolidation', 'home_improvement',
       'house', 'major_purchase', 'medical', 'moving', 'other',
       'renewable_energy', 'small_business', 'vacation'], dtype='<U18')

## Interactive Dashboard <a class="anchor" id="second-bullet"></a>

In [11]:
sc_geo = bqplot.AlbersUSA()
state_data = bqplot.topo_load('map_data/USStatesMap.json')

def_tt = bqplot.Tooltip(fields=['id', 'name'])

states_map = bqplot.Map(map_data=state_data, scales={'projection':sc_geo}, tooltip=def_tt)
states_map.interactions = {'click': 'select', 'hover': 'tooltip'}

ids, state_names = get_ids_and_names(states_map)

fig=bqplot.Figure(marks=[states_map], title='US States Map Example',
                  fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0})


# lets also make a Bar plot
x_scl = bqplot.OrdinalScale() 
y_scl = bqplot.LinearScale()
ax_xcl = bqplot.Axis(label='Purpose of Loan', scale=x_scl, tick_rotate=90)
ax_ycl = bqplot.Axis(label='Amount of Loan Issued in', 
                     scale=y_scl, 
                    orientation='vertical', side='left')
lines = bqplot.Bars(x = purpose1, y = np.zeros(len(purpose)),
                    scales = {'x': x_scl, 'y': y_scl})
fig_lines = bqplot.Figure(marks = [lines], 
                          axes = [ax_ycl, ax_xcl],)
fig_lines.axes[1].tick_style = {'text-anchor': 'start'}

# let do something additive for all states selected
def get_data_value(change):
    exports = np.zeros(len(purpose))
    snames = ''
    if change['owner'].selected is not None:
        for i,s in enumerate(change['owner'].selected):
            sn = state_names[s == ids][0]
            snames += sn + ', '
            # because of formatting, things are in arrays hence [0]
            #  also, take out state name hence [1:]
            exports_in=table.loc[table['state'] == sn].values[0][1:]
            exports = np.add(exports, exports_in.astype('float64'))
        lines.y = exports
        ax_ycl.label='Amount of Loan Issued in ' + snames
    else:
        lines.y = np.zeros(len(exports))
        ax_ycl.label='Amount of Loan Issued in NA'

states_map.observe(get_data_value,'selected')


ipywidgets.HBox([fig,fig_lines])

## Explaination and Further Scope <a class="anchor" id="third-bullet"></a>

The above dashboard is a state-wise visualization of the purpose due to which people are taking peer to peer loans. By clicking on a particular state, you can obtain the corresponding data for each state. The visualization is a bar graph since a component of the data is categorical. It is important to pay attention to the Y-axis as it is dynamic and the values are changing from state to state.

This graph can be used quite easily. A user need to hover over a state, and a dialog box will appear showing the state name and ID. If they click on the state they want to select, it will be highlighted in red color and its data will be reflected on the right side as a bar graph. <br>
Please keep in mind that if one state is selected and you click on another state, aggregate(sum) data will be represented on the bar graph as represented by the Y-axis label. To unselect a state whose data you do not want to see, simply click on it when its red so that it will be unselected and revert back to green

### Supplementory Dataset

[House Loan Dataset](https://www.kaggle.com/datasets/urstrulyvikas/house-loan-data-analysis)

This data set can be used to understand consumer behavior and the customer when they obtain house loans. Since this is the biggest category of loans underwritten in the US it can be a major business opportunity for the company to target customers and increase revenue.